In [1]:
import random
random.seed(99)

In [2]:
def reverse_complement(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement[base] for base in reversed(seq))

In [3]:
import random
import string

def random_base():
    return random.choice(['A', 'T', 'C', 'G'])

def random_quality(quality_range=40):
    # Assuming Phred+33 encoding for quality scores
    return chr(random.randint(33, 33 + quality_range - 1))

def random_string(length):
    return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(length))

def generate_fastq_record(seq_length=100):
    # Generate random sequence
    sequence = ''.join(random_base() for _ in range(seq_length))

    # Generate random quality scores
    quality = ''.join(random_quality() for _ in range(seq_length))

    # Generate random identifier
    read_name = random_string(10)

    return read_name, sequence, quality

# if __name__ == "__main__":
#     print(generate_fastq_record())


In [4]:
def read_fasta(path):
    with open(path) as f:
        for line in f:
            name = line[1:-1]
            fq = next(fq)[:-1]
            yield name, fq

In [5]:
fix1_list = ["GTCACGGGTATATGAG",
"GTCACGGGTATATGGA",
"GTCACGGGTATCACCA",
"GTCACGGGTATCAGTC",
"GTCACGGGTATCGCAT",
"GTCACGGGTATCTGCA",
"GTCACGGGTATGAAAC",
"GTCACGGGTATGAATG",
"GTCACGGGTCAAACTC",
"GTCACGGGTCAAAGAT"]

In [6]:
fix2_list = [
 "GTCATTTAGAACAACT",
"GTCATTTAGAACAATC",
"GTCATTTAGAACTCGG",
"GTCATTTAGAACTGTA",
"GTCATTTAGAAGAAGC",
"GTCATTTAGAAGATTC",
"GTCATTTAGAAGCCCA",
"GTCATTTAGAATAGGG",
"GTCATTTAGAATCTCC",
"GTCATTTAGAATGTGT",
"GTCATTTAGAATGTTG",
"GTCATTTAGAATTCCC"
]

In [7]:
barcode_list = [
"AAACCTGAGAAACCAT",
"AAACCTGAGAAACCGC",
"AAACCTGAGAAACCTA",
"AAACCTGAGAAACGAG",
"AAACCTGAGAAACGCC",
"AAACCTGAGAAAGTGG",
"AAACCTGAGAACAACT",
"AAACCTGAGAACAATC",
"AAACCTGAGAACTCGG",
]

In [8]:
read_len = 150

In [9]:
seq_info_list = [
    {"type": "Fix", "seq_list": barcode_list, "pos":0},
    {"type": "Variable"},
    {"type": "Fix", "seq_list": fix1_list, "pos":40},
    {"type": "Variable"},
    {"type": "Fix", "seq_list": fix2_list, "pos":80},
]

In [10]:
fasta_list = barcode_list + fix1_list + fix2_list

In [11]:
def get_random_pos(pos, range_len=5):
    return random.randint(max(0, pos-range_len), min(pos + range_len, read_len))

In [16]:
def read_with_block(seq_info_list):
    # block_info_list = []
    read_name, seq, qual = generate_fastq_record(seq_length=150)
    seq_list = list(seq)
    block_pos_dict = {}
    for idx, seq_info in enumerate(seq_info_list):
        seq_type = seq_info["type"]
        idx = f"{seq_type}_{idx}"
        if seq_type == "Variable":
            block_info_list = [f"{seq_type}_{idx}", seq_type, ""]
            continue
        # if idx not in block_info_dict:
        #     block_info_dict[idx] = {"fasta_seq_id": [], "type": seq_type}

        subseq_list = seq_info["seq_list"]
        subseq = random.choice(subseq_list)
        # if subseq not in block_info_dict[idx]["fasta_seq_id"]:
        #     block_info_dict[idx]["fasta_seq_id"].append(subseq)
        subseq_len = len(subseq)
        start = get_random_pos(seq_info["pos"])
        end = start + subseq_len
        seq_list[start: end] = list(subseq)
        block_pos_dict[subseq] = [start, end]
    seq_str = "".join(seq_list)
    return read_name, seq_str, qual, block_pos_dict
        # block_info_list = [f"{seq_type}_{idx}", seq_type, ""]


In [17]:
import pickle

In [26]:
with open("data/test.fastq", "w") as f, open("data/test.pos.pkl", "wb") as pkl:
    read_block_pos_dict = {}
    for ii in range(100):
        read = f"read{ii:03d}"
        _, seq, qual, block_pos_dict = read_with_block(seq_info_list)
        if ii % 2 == 0:
            seq = reverse_complement(seq)
        
        line = f"@{read}\n{seq}\n+\n{qual}\n"
        f.write(line)
        read_block_pos_dict[read] = block_pos_dict
    pickle.dump(read_block_pos_dict, pkl)

In [25]:
reverse_complement("GTCATTTAGAATAGGG")

'CCCTATTCTAAATGAC'

In [27]:
read_block_pos_dict["read001"]["AAACCTGAGAACTCGG"]

[0, 16]

## write the blockinfo

In [25]:
fasta_list =[]

In [100]:
block_info_list = []
for idx, seq_info in enumerate(seq_info_list):
    seq_type = seq_info["type"]
    idx = f"{seq_type}_{idx}"
    # block_info_list.append()
    if seq_type == "Variable":
        block_info_list.append([f"{idx}", seq_type, ""])
        continue
    fasta_id_list = ",".join(seq_info["seq_list"])
    block_info_list.append([f"{idx}", seq_type, fasta_id_list])

In [101]:
block_info_df = pd.DataFrame(block_info_list, 
        columns=["idx", "seq_type", "fasta_seq_id"])

In [102]:
block_info_df["max_mismatch"] = 2
block_info_df["query_start"] = ""
block_info_df["query_end"] = ""
block_info_df["seq_len"] = ""
block_info_df["method"] = "ANT"

In [103]:
block_info_df.to_csv("data/blockinfo.tsv", sep="\t", index=False)

In [89]:
fasta_list = barcode_list + fix1_list + fix2_list

In [92]:
with open("data/test.fasta", "w") as f:
    for xx in fasta_list:
        line = f">{xx}\n{xx}\n"
        f.write(line)

In [ ]:
# fasta_file